In [ ]:
from os import name
from numpy.core.records import array
import selenium
from selenium import webdriver as wb
import numpy as np
from bs4 import BeautifulSoup as bs
import pandas as pd



In [ ]:
def func(product_prices): #cleaning results
    nlist = []
    num = []
    for word in product_prices:
        word = word.text
        word = word.replace(',', '')
        word = word.replace('(', '')
        word = word.replace(')', '')
        for char in word.split():
            if char.isdigit():
                num.append(int(char))
    return num

In [ ]:
search = input("Enter product you want to get results for: ")
search = search.replace(' ', '+')
search = 'https://www.daraz.pk/catalog/?q=' + search + '&_keyori=ss&from=input&spm=a2a0e.searchlist.search.go.46d5287dYJtbMo'

webD = wb.Chrome(executable_path='chromedriver.exe')#chromedriver path
webD.get(search)
soup = bs(webD.page_source, 'lxml')
no_items = webD.find_element_by_xpath('//*[@id="root"]/div/div[2]/div[1]/div/div[1]/div[1]/div/div[1]/div/div')
print('Number of Items: ', (no_items.text.split()[0]))  
product_prices = webD.find_elements_by_class_name('c3gUW0')
avg = np.mean(func(product_prices))
print("Average Price is: ", round(avg,1))  # avg price calculated
reviews = webD.find_elements_by_class_name('c3XbGJ')
avgrev = np.mean(func(reviews))
print("Average Reviews: ", round(avgrev,1))  # avg reviews calculated
cnt = 0
ratinglist = []

for foo in soup.find_all('div', attrs={'class': 'c15YQ9'}):
    st = str(foo)
    cnt = st.count('c3dn4k c3EEAg')
    if (((cnt<5)==1) & ((st.count('c3dn4k c1dtTC'))==0)):
        cnt = cnt+0.5
    ratinglist.append(cnt)

avgratinglist = [x for x in ratinglist if x != 0.0]
avgratinglist = (sum(avgratinglist))/(len(avgratinglist))
print('Average Rating: ', round(avgratinglist,1))
names = webD.find_elements_by_class_name('c16H9d')
dmall = []
for foo in soup.find_all('div', attrs={'class': 'c3vCyH'}):
    st = str(foo)
    dmall.append(st.count('ic-dynamic-badge ic-dynamic-badge-lazMall ic-dynamic-group-1'))
cnt=0
original_price = []

for foo in soup.find_all('div', attrs={'class': 'c3lr34'}):
    st = str(foo)
    if((st.count('c13VH6'))==1):
        original_price.append(foo.find('del',attrs={'class':'c13VH6'}))
    else:
        original_price.append(product_prices[cnt])
    cnt=cnt+1

seller_countries = webD.find_elements_by_class_name('c2i43- ')
original_price = func(original_price[0:5])
product_prices = func(product_prices[0:5])
ratinglist = ratinglist[0:5]
reviews = func(reviews[0:5])
original_price = (original_price[0:5])
mall = []


for x in dmall:
    if x == 1:
        mall.append('Yes')
    else:
        mall.append('No')

mall = mall[0:5]
#saving results to dataframe
Daraz = {'Name': [names[0].text, names[1].text, names[2].text, names[3].text, names[4].text],
         'Daraz Mall?': mall,
         'Original Price': original_price,  # need to fix this
         'Discounted Price': product_prices,
         'Rating': ratinglist,
         'Reviews': reviews,
         'Seller Country': [seller_countries[0].text, seller_countries[1].text, seller_countries[2].text, seller_countries[3].text, seller_countries[4].text]
         }

webD.close()#closing webdriver


In [ ]:
df = pd.DataFrame(Daraz, columns=['Name', 'Daraz Mall?', 'Original Price', 'Discounted Price', 'Rating', 'Reviews', 'Seller Country'])
#print(df)
df.to_csv('result.csv',index=False)#saving result to csv file